<center><a href="https://www.rbc.com/" ><img src="https://golden-storage-production.s3.amazonaws.com/topic_images/71c114ebf3b14f51846444bd69926a3f.png" alt="RBC Logo" /></a></center>


# LLM Fine Tuning - "Queue - Assistance"

In [132]:
from DataPrep.fine_tuning_data_prep_analysis import complete_data_prep_analysis
from dotenv import load_dotenv
import os
import json
import openai
import os
from pprint import pprint
from serpapi import GoogleSearch


In [30]:
# make sure to use the latest version of the openai python package

^C


In [2]:
# Get all the API's Setup
# Reference code - https://cookbook.openai.com/examples/how_to_finetune_chat_models

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
serp_api_secret = os.getenv("SERP_API_KEY")


In [6]:
import os
from openai import OpenAI

client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY", openai_api_key))


### Training & Validation dataset

In [8]:
training_file_name = './queue/training.jsonl'
validation_file_name = './queue/validation.jsonl'

# Perform data prep and analysis - Token Counts, and file validation
print("-------Training Data Prep Analysis-------")
complete_data_prep_analysis(training_file_name)


-------Training Data Prep Analysis-------
Num examples: 28
First example:
{'role': 'system', 'content': "You will be provided queries from 'Direct Investing' customer service department at RBC. Classify each query to the right queue category.\nProvide your output in a bullet point format with only one: Queue Category\nQueue Categories are:\n--Queue 1: Account Management\n--Queue 2: Fund Transfer\n--Queue 3: External Transfers\n--Queue 4: Trading and Withdrawals"}
{'role': 'user', 'content': "I've forgotten my username for RBC Direct Investing. How can I retrieve it?"}
{'role': 'assistant', 'content': 'Queue 1: Account Management'}
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 113, 125
mean / median: 117.75, 118.0
p5 / p95: 114.0, 121.3

#### Distribution of num_assistant_to

In [9]:
print("-------Validation Data Prep Analysis-------")
complete_data_prep_analysis(validation_file_name)

-------Validation Data Prep Analysis-------
Num examples: 12
First example:
{'role': 'system', 'content': "You will be provided queries from 'Direct Investing' customer service department at RBC. Classify each query to the right queue category.\nProvide your output in a bullet point format with only one: Queue Category\nQueue Categories are:\n--Queue 1: Account Management\n--Queue 2: Fund Transfer\n--Queue 3: External Transfers\n--Queue 4: Trading and Withdrawals"}
{'role': 'user', 'content': 'How do I update my email address on my RBC Direct Investing account?'}
{'role': 'assistant', 'content': 'Queue 1: Account Management'}
No errors found
Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 114, 124
mean / median: 118.75, 118.5
p5 / p95: 114.2, 123.0

#### Distribution of num_assistant_tokens_

#### Upload Files

In [9]:
# Upload the training and validation dataset files to Azure OpenAI with the SDK.

with open(training_file_name, "rb") as training_fd:
    training_response = client.files.create(
        file=training_fd, purpose="fine-tune"
    )

training_file_id = training_response.id

with open(validation_file_name, "rb") as validation_fd:
    validation_response = client.files.create(
        file=validation_fd, purpose="fine-tune"
    )
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-vlCuB0JaCNPNBiJrsAcKKE7P
Validation file ID: file-vdvAGKV5odg3r4HSOJRTAOvy


In [13]:
### Adjust hyperparameters as needed
'''   client.fine_tuning.jobs.create(
  training_file="file-abc123", 
  model="gpt-35-turbo-0613", # Enter base model name. Note that in Azure OpenAI the model name contains dashes and cannot contain dot/period characters. 
  hyperparameters={
    "n_epochs":2,
    "learning_rate_multiplier": 2 ,
    "batch_size": 1,
  }
)   '''

##### Fine Tuning

In [11]:
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo-0125",
    suffix="queue-assist",
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-a94UB9FZ9B0jHW0cEenehRem
Status: validating_files


In [16]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)


Job ID: ftjob-a94UB9FZ9B0jHW0cEenehRem
Status: succeeded
Trained Tokens: 9723


In [17]:
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 69/84: training loss=0.00, validation loss=0.00
Step 70/84: training loss=0.00, validation loss=0.00
Step 71/84: training loss=0.00, validation loss=0.00
Step 72/84: training loss=0.00, validation loss=0.00
Step 73/84: training loss=0.00, validation loss=0.00
Step 74/84: training loss=0.00, validation loss=0.00
Step 75/84: training loss=0.00, validation loss=0.00
Step 76/84: training loss=0.00, validation loss=0.00
Step 77/84: training loss=0.00, validation loss=0.00
Step 78/84: training loss=0.00, validation loss=0.00
Step 79/84: training loss=0.00, validation loss=0.00
Step 80/84: training loss=0.00, validation loss=0.00
Step 81/84: training loss=0.00, validation loss=0.00
Step 82/84: training loss=0.00, validation loss=0.00
Step 83/84: training loss=0.00, validation loss=0.00
Step 84/84: training loss=0.00, validation loss=0.00, full validation loss=0.00
Checkpoint created at step 28 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal:queue-assist:9G69Yuvf:ckpt-step-28
Checkpoint 

In [185]:
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None: 
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal:di-txn-assist:9GCDhNRR


In [186]:
system_message = "You will be provided queries from 'Direct Investing' customer service department at RBC. Classify each query to the right queue category.\nProvide your output in a bullet point format with only one: Queue Category\nQueue Categories are:\n--Queue 1: Account Management\n--Queue 2: Fund Transfer\n--Queue 3: External Transfers\n--Queue 4: Trading and Withdrawals"
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = "What should I do if I suspect unauthorized access to my RBC Direct Investing account?"
test_messages.append({"role": "user", "content": user_message})

pprint(test_messages)

[{'content': "You will be provided queries from 'Direct Investing' customer "
             'service department at RBC. Classify each query to the right '
             'queue category.\n'
             'Provide your output in a bullet point format with only one: '
             'Queue Category\n'
             'Queue Categories are:\n'
             '--Queue 1: Account Management\n'
             '--Queue 2: Fund Transfer\n'
             '--Queue 3: External Transfers\n'
             '--Queue 4: Trading and Withdrawals',
  'role': 'system'},
 {'content': 'What should I do if I suspect unauthorized access to my RBC '
             'Direct Investing account?',
  'role': 'user'}]


In [187]:
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=50
)
print(response.choices[0].message.content)

Queue 1: Account Management


# LLM Fine Tuning - "QnA Direct Investing Assistance"

#### Transfer learning within Queue Assistance Fine Tune LLM Model

In [89]:
training_file_name = './DI_QnA_Assistance/training_DI_QnA.jsonl'
validation_file_name = './DI_QnA_Assistance/validation_DI_QnA.jsonl'

# Perform data prep and analysis - Token Counts, and file validation
print("-------Training Data Prep Analysis-------")
complete_data_prep_analysis(training_file_name)


-------Training Data Prep Analysis-------
Num examples: 10
First example:
{'role': 'system', 'content': "You will be provided queries from customer service department. Your task is to answer the question using only the provided steps with reference at the end. If the document does not contain the information needed to answer this question then write: 'Insufficient information, please contact RBC Direct Investing customer service representative at 1-800-769-2560.' Reference link should be in the format: [Title](URL)\nNOTE: KEEP THE URL SAME AS GIVEN IN EACH QUESTION END}"}
{'role': 'user', 'content': 'How do I update my address with RBC Direct Investing?'}
{'role': 'assistant', 'content': "To update your address with RBC Direct Investing, follow these steps:\n1. Log in to your RBC Direct Investing account.\n2. Navigate to the 'Update Profile' section.\n3. Select 'Change your Account Profile' and complete the form with your new address details.\n4. Click 'Continue' to submit the request.

In [91]:
complete_data_prep_analysis(validation_file_name)

Num examples: 3
First example:
{'role': 'system', 'content': "You will be provided queries from customer service department. Your task is to answer the question using only the provided steps with reference at the end. If the document does not contain the information needed to answer this question then write: 'Insufficient information, please contact RBC Direct Investing customer service representative at 1-800-769-2560.' Reference link should be in the format: [Title](URL)\nNOTE: KEEP THE LINK SAME AS GIVEN IN EACH QUESTION END}"}
{'role': 'user', 'content': 'How do I open a new RRSP?'}
{'role': 'assistant', 'content': 'To open a new RRSP, follow these steps:\n1. Call 1-800-463-3863 and select the Investments option before midnight (local time) of March 1, 2010. Our investment specialists are available by phone 24 hours a day, 7 days a week.\n2. Visit your local RBC Royal Bank branch by the close of the March 1, 2010 business day. Schedule an appointment by calling ahead.\nReference: [

In [92]:
# Upload the training and validation dataset files to Azure OpenAI with the SDK.

with open(training_file_name, "rb") as training_fd:
    training_response = client.files.create(
        file=training_fd, purpose="fine-tune"
    )

training_file_id = training_response.id

with open(validation_file_name, "rb") as validation_fd:
    validation_response = client.files.create(
        file=validation_fd, purpose="fine-tune"
    )
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-P5niTGQG4Y03ls6Np5FKRsg5
Validation file ID: file-BpNfkOb50i2SUCnmH7mKvI85


In [93]:
response = client.fine_tuning.jobs.create(
  training_file=training_file_id,
  validation_file=validation_file_id,
  hyperparameters={
    "n_epochs": 3,
    "learning_rate_multiplier": 3,
    "batch_size": 1,
  },
  model=fine_tuned_model_id,
  suffix="QnA-Assist-DI",
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-5oXKu7dnBOXz3hWDOYdBztvT
Status: validating_files


In [98]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-5oXKu7dnBOXz3hWDOYdBztvT
Status: succeeded
Trained Tokens: 9939


In [99]:
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 15/30: training loss=0.80, validation loss=0.90
Step 16/30: training loss=0.97, validation loss=0.97
Step 17/30: training loss=0.82, validation loss=1.32
Step 18/30: training loss=0.77, validation loss=0.92
Step 19/30: training loss=0.30, validation loss=0.97
Step 20/30: training loss=0.88, validation loss=1.35, full validation loss=1.01
Step 21/30: training loss=1.12, validation loss=0.94
Step 22/30: training loss=0.84, validation loss=1.00
Step 23/30: training loss=0.22, validation loss=1.39
Step 24/30: training loss=0.71, validation loss=0.96
Step 25/30: training loss=0.70, validation loss=1.03
Step 26/30: training loss=0.84, validation loss=1.41
Step 27/30: training loss=0.57, validation loss=0.98
Step 28/30: training loss=0.66, validation loss=1.04
Step 29/30: training loss=0.83, validation loss=1.41
Step 30/30: training loss=0.86, validation loss=0.98, full validation loss=1.05
Checkpoint created at step 10 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal:qna-assist-di:9GAwM

In [100]:
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None: 
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal:qna-assist-di:9GAwMxch


In [104]:
system_message = "You will be provided queries from customer service department. Your task is to answer the question using only the provided steps with reference at the end. If the document does not contain the information needed to answer this question then write: 'Insufficient information, please contact RBC Direct Investing customer service representative at 1-800-769-2560."
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = "How can I place a trade with RBC Direct Investing account?"
test_messages.append({"role": "user", "content": user_message})

pprint(test_messages)

[{'content': 'You will be provided queries from customer service department. '
             'Your task is to answer the question using only the provided '
             'steps with reference at the end. If the document does not '
             'contain the information needed to answer this question then '
             "write: 'Insufficient information, please contact RBC Direct "
             'Investing customer service representative at 1-800-769-2560.',
  'role': 'system'},
 {'content': 'How can I place a trade with RBC Direct Investing account?',
  'role': 'user'}]


In [105]:
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=200
)
print(response.choices[0].message.content)

To place a trade with RBC Direct Investing account, follow these steps:
1. Log in to your RBC Direct Investing account.
2. Click on the 'Trading' tab.
3. Select 'Order Entry' from the drop-down menu.
4. Enter the required information for your trade, including the symbol, quantity, and order type.
5. Review the order details and click 'Submit Order' to complete the trade.
6. Note: If you are placing a trade outside of regular market hours, ensure you have selected the appropriate order type (e.g., 'Extended Hours' or 'Good 'til Canceled').
7. If you need assistance with placing a trade, contact RBC Direct Investing customer service at 1-800-769-2560.
[Reference](https://www.rbcroyalbank.com/onlinebanking/bankingusertips/investments/adirect/trade.html)


In [183]:
system_message = "You will be provided queries from 'Direct Investing' customer service department at RBC. Classify each query to the right queue category.\nProvide your output in a bullet point format with only one: Queue Category\nQueue Categories are:\n--Queue 1: Account Management\n--Queue 2: Fund Transfer\n--Queue 3: External Transfers\n--Queue 4: Trading and Withdrawals"
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = "How do I update my address with RBC Direct Investing?"
test_messages.append({"role": "user", "content": user_message})

pprint(test_messages)
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=50
)
print(response.choices[0].message.content)


[{'content': "You will be provided queries from 'Direct Investing' customer "
             'service department at RBC. Classify each query to the right '
             'queue category.\n'
             'Provide your output in a bullet point format with only one: '
             'Queue Category\n'
             'Queue Categories are:\n'
             '--Queue 1: Account Management\n'
             '--Queue 2: Fund Transfer\n'
             '--Queue 3: External Transfers\n'
             '--Queue 4: Trading and Withdrawals',
  'role': 'system'},
 {'content': 'How do I update my address with RBC Direct Investing?',
  'role': 'user'}]
Queue 1: Account Management


# LLM Fine Tuning - "Transactional Assistance"

In [108]:
training_file_name = './Transactional/training_DI_txn.jsonl'
validation_file_name = './Transactional/validation_DI_txn.jsonl'

# Perform data prep and analysis - Token Counts, and file validation
print("-------Training Data Prep Analysis-------")
complete_data_prep_analysis(training_file_name)


-------Training Data Prep Analysis-------
Num examples: 13
First example:
{'role': 'system', 'content': "You are given with the transactional data and your task is to process this tabular data containing both text and numerical values based on criteria then generate answers.\n-The answers should be in bullets and sub-bullet points\n-If the answer cannot be found in the Transactional data given below, write 'please contact RBC Direct Investing Team.'\nTransactional Data :\n| Account No | Request Date | Transactional Details | Transfer Date | Withdrawal Amount | Deposit Amount | Balance Amount | Expected Transfer Days | Status | Note | Last Note Date |\n|------------|--------------|---------------------|---------------|----------------|-------------|-------------|-----------------------|--------|------|----------------|\n| 123456 | 2024-01-05 | Transferred cash from TD Bank to RBC | 2024-02-14 | - | 75000 | 150000 | 10 | Opened | Contacted customer to confirm transaction | 2024-02-14 |\n

In [109]:
complete_data_prep_analysis(validation_file_name)

Num examples: 6
First example:
{'role': 'system', 'content': "You are given with the transactional data and your task is to process this tabular data containing both text and numerical values based on criteria then generate answers.\n-The answers should be in bullets and sub-bullet points\n-If the answer cannot be found in the Transactional data given below, write 'please contact RBC Direct Investing Team.'\nTransactional Data :\n| Account No | Request Date | Transactional Details | Transfer Date | Withdrawal Amount | Deposit Amount | Balance Amount | Expected Transfer Days | Status | Note | Last Note Date |\n|------------|--------------|---------------------|---------------|----------------|-------------|-------------|-----------------------|--------|------|----------------|\n| 123456 | 2024-01-05 | Transferred cash from TD Bank to RBC | 2024-02-14 | - | 75000 | 150000 | 10 | Opened | Contacted customer to confirm transaction | 2024-02-14 |\n| 789012 | 2024-01-10 | Transferred funds f

In [110]:
# Upload the training and validation dataset files to Azure OpenAI with the SDK.

with open(training_file_name, "rb") as training_fd:
    training_response = client.files.create(
        file=training_fd, purpose="fine-tune"
    )

training_file_id = training_response.id

with open(validation_file_name, "rb") as validation_fd:
    validation_response = client.files.create(
        file=validation_fd, purpose="fine-tune"
    )
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-ebKtNr4G1PPX7aEFfs5DD1E0
Validation file ID: file-9CoT1nWGFJXS78zzQ7fO7HFO


In [111]:
response = client.fine_tuning.jobs.create(
  training_file=training_file_id,
  validation_file=validation_file_id,
  hyperparameters={
    "n_epochs": 3,
    "learning_rate_multiplier": 2,
    "batch_size": 1,
  },
  model=fine_tuned_model_id,
  suffix="DI-Txn-Assist",
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-BKhVDlxr13kb7utrdB7zh8b8
Status: validating_files


In [122]:
fine_tuned_model_id

'ft:gpt-3.5-turbo-0125:personal:di-txn-assist:9GCDhNRR'

In [119]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-BKhVDlxr13kb7utrdB7zh8b8
Status: succeeded
Trained Tokens: 27996


In [120]:
response = client.fine_tuning.jobs.list_events(job_id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 24/39: training loss=0.26, validation loss=0.64
Step 25/39: training loss=0.27, validation loss=0.01
Step 26/39: training loss=0.14, validation loss=0.12, full validation loss=0.17
Step 27/39: training loss=0.01, validation loss=0.66
Step 28/39: training loss=0.05, validation loss=0.18
Step 29/39: training loss=0.08, validation loss=0.11
Step 30/39: training loss=0.06, validation loss=0.66
Step 31/39: training loss=0.17, validation loss=0.00
Step 32/39: training loss=0.08, validation loss=0.07
Step 33/39: training loss=0.04, validation loss=0.64
Step 34/39: training loss=0.11, validation loss=0.12
Step 35/39: training loss=0.62, validation loss=0.06
Step 36/39: training loss=0.00, validation loss=0.55
Step 37/39: training loss=0.03, validation loss=0.00
Step 38/39: training loss=0.05, validation loss=0.05
Step 39/39: training loss=0.56, validation loss=0.55, full validation loss=0.12
Checkpoint created at step 13 with Snapshot ID: ft:gpt-3.5-turbo-0125:personal:di-txn-assist:9GCDg

In [184]:
response = client.fine_tuning.jobs.retrieve(job_id)
fine_tuned_model_id = response.fine_tuned_model

if fine_tuned_model_id is None: 
    raise RuntimeError("Fine-tuned model ID not found. Your job has likely not been completed yet.")

print("Fine-tuned model ID:", fine_tuned_model_id)

Fine-tuned model ID: ft:gpt-3.5-turbo-0125:personal:di-txn-assist:9GCDhNRR


In [123]:
system_message = "You are given with the transactional data and your task is to process this tabular data containing both text and numerical values based on criteria then generate answers.\n-The answers should be in bullets and sub-bullet points\n-If the answer cannot be found in the Transactional data given below, write 'please contact RBC Direct Investing Team.'\nTransactional Data :\n| Account No | Request Date | Transactional Details | Transfer Date | Withdrawal Amount | Deposit Amount | Balance Amount | Expected Transfer Days | Status | Note | Last Note Date |\n|------------|--------------|---------------------|---------------|----------------|-------------|-------------|-----------------------|--------|------|----------------|\n| 123456 | 2024-01-05 | Transferred cash from TD Bank to RBC | 2024-02-14 | - | 75000 | 150000 | 10 | Opened | Contacted customer to confirm transaction | 2024-02-14 |\n| 789012 | 2024-01-10 | Transferred funds from CIBC to RBC | 2024-01-25 | - | 100000 | 200000 | 15 | Opened | Resolved missing withdrawal transaction information | 2024-01-25 |\n| 345678 | 2024-01-15 | Withdrew cash for investment | 2024-02-05 | 50000 | - | 150000 | - | Closed | - | - |\n| 901234 | 2024-01-20 | Deposited funds from BMO into account | 2024-02-07 | - | 85000 | 185000 | 18 | Opened | Verified deposit with customer | 2024-02-07 |\n| 567890 | 2024-01-25 | Transferred money from RBC to TD Bank | 2024-02-10 | - | 60000 | 160000 | 16 | Opened | Sent confirmation email to customer | 2024-02-10 |\n| 234567 | 2024-02-03 | Withdrew funds for stock purchase | 2024-02-25 | 70000 | - | 160000 | - | Closed | - | - |\n| 890123 | 2024-02-08 | Transferred cash from ScotiaBank to RBC | 2024-02-28 | - | 95000 | 195000 | 20 | Opened | Called customer on the same day | 2024-02-28 |\n| 456789 | 2024-02-12 | Deposited funds from RBC into account | 2024-03-06 | - | 80000 | 180000 | 22 | Opened | Resolved deposit discrepancy | 2024-03-06 |\n| 123456 | 2024-02-18 | Withdrew cash for investment | 2024-03-12 | 45000 | - | 135000 | - | Closed | -"
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = "List all accounts with a status of 'Closed' and a balance amount greater than $150,000"
test_messages.append({"role": "user", "content": user_message})

pprint(test_messages)

[{'content': 'You are given with the transactional data and your task is to '
             'process this tabular data containing both text and numerical '
             'values based on criteria then generate answers.\n'
             '-The answers should be in bullets and sub-bullet points\n'
             '-If the answer cannot be found in the Transactional data given '
             "below, write 'please contact RBC Direct Investing Team.'\n"
             'Transactional Data :\n'
             '| Account No | Request Date | Transactional Details | Transfer '
             'Date | Withdrawal Amount | Deposit Amount | Balance Amount | '
             'Expected Transfer Days | Status | Note | Last Note Date |\n'
             '|------------|--------------|---------------------|---------------|----------------|-------------|-------------|-----------------------|--------|------|----------------|\n'
             '| 123456 | 2024-01-05 | Transferred cash from TD Bank to RBC | '
             '2024-

In [124]:
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=200
)
print(response.choices[0].message.content)

Account: 234567
Balance Amount: $160,000
Withdrawal Amount: $70,000
++++++++++++++++++
Account: 789012
Balance Amount: $200,000
Withdrawal Amount: $100,000


In [125]:
system_message = "You are given with the transactional data and your task is to process this tabular data containing both text and numerical values based on criteria then generate answers.\n-The answers should be in bullets and sub-bullet points\n-If the answer cannot be found in the Transactional data given below, write 'please contact RBC Direct Investing Team.'\nTransactional Data :\n| Account No | Request Date | Transactional Details | Transfer Date | Withdrawal Amount | Deposit Amount | Balance Amount | Expected Transfer Days | Status | Note | Last Note Date |\n|------------|--------------|---------------------|---------------|----------------|-------------|-------------|-----------------------|--------|------|----------------|\n| 123456 | 2024-01-05 | Transferred cash from TD Bank to RBC | 2024-02-14 | - | 75000 | 150000 | 10 | Opened | Contacted customer to confirm transaction | 2024-02-14 |\n| 789012 | 2024-01-10 | Transferred funds from CIBC to RBC | 2024-01-25 | - | 100000 | 200000 | 15 | Opened | Resolved missing withdrawal transaction information | 2024-01-25 |\n| 345678 | 2024-01-15 | Withdrew cash for investment | 2024-02-05 | 50000 | - | 150000 | - | Closed | - | - |\n| 901234 | 2024-01-20 | Deposited funds from BMO into account | 2024-02-07 | - | 85000 | 185000 | 18 | Opened | Verified deposit with customer | 2024-02-07 |\n| 567890 | 2024-01-25 | Transferred money from RBC to TD Bank | 2024-02-10 | - | 60000 | 160000 | 16 | Opened | Sent confirmation email to customer | 2024-02-10 |\n| 234567 | 2024-02-03 | Withdrew funds for stock purchase | 2024-02-25 | 70000 | - | 160000 | - | Closed | - | - |\n| 890123 | 2024-02-08 | Transferred cash from ScotiaBank to RBC | 2024-02-28 | - | 95000 | 195000 | 20 | Opened | Called customer on the same day | 2024-02-28 |\n| 456789 | 2024-02-12 | Deposited funds from RBC into account | 2024-03-06 | - | 80000 | 180000 | 22 | Opened | Resolved deposit discrepancy | 2024-03-06 |\n| 123456 | 2024-02-18 | Withdrew cash for investment | 2024-03-12 | 45000 | - | 135000 | - | Closed | -"
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = "Give me all accounts where the 'Expected Transfer Days' is greater than 20"
test_messages.append({"role": "user", "content": user_message})

pprint(test_messages)

[{'content': 'You are given with the transactional data and your task is to '
             'process this tabular data containing both text and numerical '
             'values based on criteria then generate answers.\n'
             '-The answers should be in bullets and sub-bullet points\n'
             '-If the answer cannot be found in the Transactional data given '
             "below, write 'please contact RBC Direct Investing Team.'\n"
             'Transactional Data :\n'
             '| Account No | Request Date | Transactional Details | Transfer '
             'Date | Withdrawal Amount | Deposit Amount | Balance Amount | '
             'Expected Transfer Days | Status | Note | Last Note Date |\n'
             '|------------|--------------|---------------------|---------------|----------------|-------------|-------------|-----------------------|--------|------|----------------|\n'
             '| 123456 | 2024-01-05 | Transferred cash from TD Bank to RBC | '
             '2024-

In [126]:
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=200
)
print(response.choices[0].message.content)

Account: 456789
Expected Transfer Days: 22
++++++++++++++++++
Account: 890123
Expected Transfer Days: 20


In [219]:
system_message = "You are given with the transactional data and your task is to process this tabular data containing both text and numerical values based on criteria then generate answers.\n-The answers should be in bullets and sub-bullet points\n-If the answer cannot be found in the Transactional data given below, write 'please contact RBC Direct Investing Team.'\nTransactional Data :\n| Account No | Request Date | Transactional Details | Transfer Date | Withdrawal Amount | Deposit Amount | Balance Amount | Expected Transfer Days | Status | Note | Last Note Date |\n|------------|--------------|---------------------|---------------|----------------|-------------|-------------|-----------------------|--------|------|----------------|\n| 123456 | 2024-01-05 | Transferred cash from TD Bank to RBC | 2024-02-14 | - | 75000 | 150000 | 10 | Opened | Contacted customer to confirm transaction | 2024-02-14 |\n| 789012 | 2024-01-10 | Transferred funds from CIBC to RBC | 2024-01-25 | - | 100000 | 200000 | 15 | Opened | Resolved missing withdrawal transaction information | 2024-01-25 |\n| 345678 | 2024-01-15 | Withdrew cash for investment | 2024-02-05 | 50000 | - | 150000 | - | Closed | - | - |\n| 901234 | 2024-01-20 | Deposited funds from BMO into account | 2024-02-07 | - | 85000 | 185000 | 18 | Opened | Verified deposit with customer | 2024-02-07 |\n| 567890 | 2024-01-25 | Transferred money from RBC to TD Bank | 2024-02-10 | - | 60000 | 160000 | 16 | Opened | Sent confirmation email to customer | 2024-02-10 |\n| 234567 | 2024-02-03 | Withdrew funds for stock purchase | 2024-02-25 | 70000 | - | 160000 | - | Closed | - | - |\n| 890123 | 2024-02-08 | Transferred cash from ScotiaBank to RBC | 2024-02-28 | - | 95000 | 195000 | 20 | Opened | Called customer on the same day | 2024-02-28 |\n| 456789 | 2024-02-12 | Deposited funds from RBC into account | 2024-03-06 | - | 80000 | 180000 | 22 | Opened | Resolved deposit discrepancy | 2024-03-06 |\n| 123456 | 2024-02-18 | Withdrew cash for investment | 2024-03-12 | 45000 | - | 135000 | - | Closed | -"
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = "Give me all accounts where the transaction detail - 'Withdrew cash for investment'"
test_messages.append({"role": "user", "content": user_message})

pprint(test_messages)

[{'content': 'You are given with the transactional data and your task is to '
             'process this tabular data containing both text and numerical '
             'values based on criteria then generate answers.\n'
             '-The answers should be in bullets and sub-bullet points\n'
             '-If the answer cannot be found in the Transactional data given '
             "below, write 'please contact RBC Direct Investing Team.'\n"
             'Transactional Data :\n'
             '| Account No | Request Date | Transactional Details | Transfer '
             'Date | Withdrawal Amount | Deposit Amount | Balance Amount | '
             'Expected Transfer Days | Status | Note | Last Note Date |\n'
             '|------------|--------------|---------------------|---------------|----------------|-------------|-------------|-----------------------|--------|------|----------------|\n'
             '| 123456 | 2024-01-05 | Transferred cash from TD Bank to RBC | '
             '2024-

In [220]:
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=200
)
print(response.choices[0].message.content)

Account: 345678
Withdrawal Amount: $50,000
Balance Amount: $150,000
Withdrawal Date: 2024-02-05
++++++++++++++++++
Account: 123456
Withdrawal Amount: $45,000
Balance Amount: $135,000
Withdrawal Date: 2024-03-12


In [129]:
system_message = "You are given with the transactional data and your task is to process this tabular data containing both text and numerical values based on criteria then generate answers.\n-The answers should be in bullets and sub-bullet points\n-If the answer cannot be found in the Transactional data given below, write 'please contact RBC Direct Investing Team.'\nTransactional Data :\n| Account No | Request Date | Transactional Details | Transfer Date | Withdrawal Amount | Deposit Amount | Balance Amount | Expected Transfer Days | Status | Note | Last Note Date |\n|------------|--------------|---------------------|---------------|----------------|-------------|-------------|-----------------------|--------|------|----------------|\n| 123456 | 2024-01-05 | Transferred cash from TD Bank to RBC | 2024-02-14 | - | 75000 | 150000 | 10 | Opened | Contacted customer to confirm transaction | 2024-02-14 |\n| 789012 | 2024-01-10 | Transferred funds from CIBC to RBC | 2024-01-25 | - | 100000 | 200000 | 15 | Opened | Resolved missing withdrawal transaction information | 2024-01-25 |\n| 345678 | 2024-01-15 | Withdrew cash for investment | 2024-02-05 | 50000 | - | 150000 | - | Closed | - | - |\n| 901234 | 2024-01-20 | Deposited funds from BMO into account | 2024-02-07 | - | 85000 | 185000 | 18 | Opened | Verified deposit with customer | 2024-02-07 |\n| 567890 | 2024-01-25 | Transferred money from RBC to TD Bank | 2024-02-10 | - | 60000 | 160000 | 16 | Opened | Sent confirmation email to customer | 2024-02-10 |\n| 234567 | 2024-02-03 | Withdrew funds for stock purchase | 2024-02-25 | 70000 | - | 160000 | - | Closed | - | - |\n| 890123 | 2024-02-08 | Transferred cash from ScotiaBank to RBC | 2024-02-28 | - | 95000 | 195000 | 20 | Opened | Called customer on the same day | 2024-02-28 |\n| 456789 | 2024-02-12 | Deposited funds from RBC into account | 2024-03-06 | - | 80000 | 180000 | 22 | Opened | Resolved deposit discrepancy | 2024-03-06 |\n| 123456 | 2024-02-18 | Withdrew cash for investment | 2024-03-12 | 45000 | - | 135000 | - | Closed | -"
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = "Give me all accounts where the transaction detail - 'Withdrew cash for investment', I need only account numbers"
test_messages.append({"role": "user", "content": user_message})

pprint(test_messages)

[{'content': 'You are given with the transactional data and your task is to '
             'process this tabular data containing both text and numerical '
             'values based on criteria then generate answers.\n'
             '-The answers should be in bullets and sub-bullet points\n'
             '-If the answer cannot be found in the Transactional data given '
             "below, write 'please contact RBC Direct Investing Team.'\n"
             'Transactional Data :\n'
             '| Account No | Request Date | Transactional Details | Transfer '
             'Date | Withdrawal Amount | Deposit Amount | Balance Amount | '
             'Expected Transfer Days | Status | Note | Last Note Date |\n'
             '|------------|--------------|---------------------|---------------|----------------|-------------|-------------|-----------------------|--------|------|----------------|\n'
             '| 123456 | 2024-01-05 | Transferred cash from TD Bank to RBC | '
             '2024-

In [130]:
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=200
)
print(response.choices[0].message.content)

Account Number: 345678
Account Number: 123456


In [131]:
## Fine tuned Trained Model - 
fine_tuned_model_id

'ft:gpt-3.5-turbo-0125:personal:di-txn-assist:9GCDhNRR'

# -----------------------------------------------------------------------------------------------------------------------------------------------

### OpenAI Functions

In [172]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def search_internet(search_query):
    """Get the latest news from the internet on Finance for given search_query"""
    if search_query == None:
        print("LLM Didn't pass the argument")
        return
    params = {
    "engine": "google",
    "q": search_query,
    "api_key": serp_api_secret
    }
    print('--------------')
    print(params)
    print('--------------')
    search = GoogleSearch(params)
    results = search.get_dict()
    print('--------------')
    print(results)
    print('--------------')
    organic_results = results["organic_results"][:3]

    # Extract 'link' and 'snippet' fields from each dictionary
    filtered_data = [{'link': item['link'], 'snippet': item['snippet']} for item in organic_results]

    # Convert the filtered data to JSON
    return json.dumps(filtered_data, indent=4)


In [225]:
system_message = """You are Fine-tuned on following 3 tasks:
1. Queue Task - You will be provided queries from 'Direct Investing' customer service department at RBC. Classify each query to the right queue category.\nProvide your output in a bullet point format with only one: Queue Category\nQueue Categories are:\n--Queue 1: Account Management\n--Queue 2: Fund Transfer\n--Queue 3: External Transfers\n--Queue 4: Trading and Withdrawals"

2. Transactional Task - You are given with the transactional data and your task is to process this tabular data containing both text and numerical values based on criteria then generate answers.\n-The answers should be in bullets and sub-bullet points\n-If the answer cannot be found in the Transactional data given below, write 'please contact RBC Direct Investing Team.'\nTransactional Data :\n| Account No | Request Date | Transactional Details | Transfer Date | Withdrawal Amount | Deposit Amount | Balance Amount | Expected Transfer Days | Status | Note | Last Note Date |\n|------------|--------------|---------------------|---------------|----------------|-------------|-------------|-----------------------|--------|------|----------------|\n| 123456 | 2024-01-05 | Transferred cash from TD Bank to RBC | 2024-02-14 | - | 75000 | 150000 | 10 | Opened | Contacted customer to confirm transaction | 2024-02-14 |\n| 789012 | 2024-01-10 | Transferred funds from CIBC to RBC | 2024-01-25 | - | 100000 | 200000 | 15 | Opened | Resolved missing withdrawal transaction information | 2024-01-25 |\n| 345678 | 2024-01-15 | Withdrew cash for investment | 2024-02-05 | 50000 | - | 150000 | - | Closed | - | - |\n| 901234 | 2024-01-20 | Deposited funds from BMO into account | 2024-02-07 | - | 85000 | 185000 | 18 | Opened | Verified deposit with customer | 2024-02-07 |\n| 567890 | 2024-01-25 | Transferred money from RBC to TD Bank | 2024-02-10 | - | 60000 | 160000 | 16 | Opened | Sent confirmation email to customer | 2024-02-10 |\n| 234567 | 2024-02-03 | Withdrew funds for stock purchase | 2024-02-25 | 70000 | - | 160000 | - | Closed | - | - |\n| 890123 | 2024-02-08 | Transferred cash from ScotiaBank to RBC | 2024-02-28 | - | 95000 | 195000 | 20 | Opened | Called customer on the same day | 2024-02-28 |\n| 456789 | 2024-02-12 | Deposited funds from RBC into account | 2024-03-06 | - | 80000 | 180000 | 22 | Opened | Resolved deposit discrepancy | 2024-03-06 |\n| 123456 | 2024-02-18 | Withdrew cash for investment | 2024-03-12 | 45000 | - | 135000 | - | Closed | -

3. Customer Service QnA Task - You will be provided queries from customer service department. Your task is to answer the question using only the provided steps with reference at the end. If the document does not contain the information needed to answer this question then write: 'Insufficient information, please contact RBC Direct Investing customer service representative at 1-800-769-2560.

--You are allowed to generate search_query text on Financial Industries such as Dividends, Earnings per share/EPS etc. and plug into functions.  Provide Answer with Reference link from function in the format: [Answer](URL)"
"""

def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = []
    messages.append({"role": "system", "content": system_message})
    messages.append({"role": "user", "content": "Scotiabank EPS for Q3 2023'"})
    tools = [
         {
        "type": "function",
        "function": {
            "name": "search_internet",
            "description": "Search the internet for the search_query on Finance.",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_query": {
                        "type": "string",
                        "description": "The query to search for on the internet."
                    }
                },
                "required": ["search_query"]
            }
        }
    }
    ]
    response = client.chat.completions.create(
        model=fine_tuned_model_id,
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    print('+++++++++++++++++++++++')
    print(response_message)
    print('+++++++++++++++++++++++')
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        print('+++++++++++Invoking function++++++++')
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "search_internet": search_internet,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            print('function to call -->',function_to_call)
            function_args = json.loads(tool_call.function.arguments)
            print('function to call w/ arguments-->',function_args)
            function_response = function_to_call(
                search_query=function_args.get("search_query")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model=fine_tuned_model_id,
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response


In [226]:
print(run_conversation().choices[0].message.content)

+++++++++++++++++++++++
ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_rscUcaFSDSfKgncojs9ashge', function=Function(arguments='{"search_query":"Scotiabank EPS Q3 2023"}', name='search_internet'), type='function')])
+++++++++++++++++++++++
+++++++++++Invoking function++++++++
function to call --> <function search_internet at 0x000002147EB580E0>
function to call w/ arguments--> {'search_query': 'Scotiabank EPS Q3 2023'}
--------------
{'engine': 'google', 'q': 'Scotiabank EPS Q3 2023', 'api_key': '4dcedd8e41021dea1a8c248e4bc07bf66eded7353ef3813cbff51d1a883490b2'}
--------------
--------------
{'search_metadata': {'id': '6624aed6216a9d1f954b49b8', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/2d498b52a215f8ae/6624aed6216a9d1f954b49b8.json', 'created_at': '2024-04-21 06:14:46 UTC', 'processed_at': '2024-04-21 06:14:46 UTC', 'google_url': 'https://www.google.com/search?q=Scotiabank+EPS+Q3+202

In [212]:
system_message = """You are Fine-tuned on following 3 tasks:
1. Queue Task - You will be provided queries from 'Direct Investing' customer service department at RBC. Classify each query to the right queue category.\nProvide your output in a bullet point format with only one: Queue Category\nQueue Categories are:\n--Queue 1: Account Management\n--Queue 2: Fund Transfer\n--Queue 3: External Transfers\n--Queue 4: Trading and Withdrawals"

2. Transactional Task - You are given with the transactional data and your task is to process this tabular data containing both text and numerical values based on criteria then generate answers.\n-The answers should be in bullets and sub-bullet points\n-If the answer cannot be found in the Transactional data given below, write 'please contact RBC Direct Investing Team.'\nTransactional Data :\n| Account No | Request Date | Transactional Details | Transfer Date | Withdrawal Amount | Deposit Amount | Balance Amount | Expected Transfer Days | Status | Note | Last Note Date |\n|------------|--------------|---------------------|---------------|----------------|-------------|-------------|-----------------------|--------|------|----------------|\n| 123456 | 2024-01-05 | Transferred cash from TD Bank to RBC | 2024-02-14 | - | 75000 | 150000 | 10 | Opened | Contacted customer to confirm transaction | 2024-02-14 |\n| 789012 | 2024-01-10 | Transferred funds from CIBC to RBC | 2024-01-25 | - | 100000 | 200000 | 15 | Opened | Resolved missing withdrawal transaction information | 2024-01-25 |\n| 345678 | 2024-01-15 | Withdrew cash for investment | 2024-02-05 | 50000 | - | 150000 | - | Closed | - | - |\n| 901234 | 2024-01-20 | Deposited funds from BMO into account | 2024-02-07 | - | 85000 | 185000 | 18 | Opened | Verified deposit with customer | 2024-02-07 |\n| 567890 | 2024-01-25 | Transferred money from RBC to TD Bank | 2024-02-10 | - | 60000 | 160000 | 16 | Opened | Sent confirmation email to customer | 2024-02-10 |\n| 234567 | 2024-02-03 | Withdrew funds for stock purchase | 2024-02-25 | 70000 | - | 160000 | - | Closed | - | - |\n| 890123 | 2024-02-08 | Transferred cash from ScotiaBank to RBC | 2024-02-28 | - | 95000 | 195000 | 20 | Opened | Called customer on the same day | 2024-02-28 |\n| 456789 | 2024-02-12 | Deposited funds from RBC into account | 2024-03-06 | - | 80000 | 180000 | 22 | Opened | Resolved deposit discrepancy | 2024-03-06 |\n| 123456 | 2024-02-18 | Withdrew cash for investment | 2024-03-12 | 45000 | - | 135000 | - | Closed | -

3. Customer Service QnA Task - You will be provided queries from customer service department. Your task is to answer the question using only the provided steps with reference at the end. If the document does not contain the information needed to answer this question then write: 'Insufficient information, please contact RBC Direct Investing customer service representative at 1-800-769-2560.

--You are allowed to generate search_query text on Financial Industries such as Dividends, Earnings per share/EPS etc. and plug into functions.  Provide Answer with Reference link from function in the format: [Answer](URL)"
"""
test_messages = []
test_messages.append({"role": "system", "content": system_message})


In [213]:
user_message = "Which queue for this task- Can I transfer funds from my RBC Direct Investing account to a non-Canadian bank account?"
test_messages.append({"role": "user", "content": user_message})

pprint(test_messages)

[{'content': 'You are Fine-tuned on following 3 tasks:\n'
             "1. Queue Task - You will be provided queries from 'Direct "
             "Investing' customer service department at RBC. Classify each "
             'query to the right queue category.\n'
             'Provide your output in a bullet point format with only one: '
             'Queue Category\n'
             'Queue Categories are:\n'
             '--Queue 1: Account Management\n'
             '--Queue 2: Fund Transfer\n'
             '--Queue 3: External Transfers\n'
             '--Queue 4: Trading and Withdrawals"\n'
             '\n'
             '2. Transactional Task - You are given with the transactional '
             'data and your task is to process this tabular data containing '
             'both text and numerical values based on criteria then generate '
             'answers.\n'
             '-The answers should be in bullets and sub-bullet points\n'
             '-If the answer cannot be found in the 

In [214]:
response = client.chat.completions.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=200
)
print(response.choices[0].message.content)

Queue 3: External Transfers
